In [2]:
import gurobipy as gp
import numpy as np
from typing import Dict, Tuple
from time import time_ns
import time
import pandas as pd

In [3]:
from pathlib import Path
import numpy as np
import json


# ---------------------------
# INPUTS (match user's simple style)
# ---------------------------
EXP_RIDERSHIP = {
    'Lakeshore West': 33239,
    'Lakeshore East': 25769,
    'Kitchener': 15071,
    'Milton': 13138,
    'Barrie': 8636,
    'Stouffville': 7341,
    'Richmond Hill': 4642
}

INTERVAL_SIZE = 5  # minutes, window is 4:00–6:00 PM (120 total)

# PROFILES:
# - Each profile has parameters tuned for realistic patterns.
# - global_scale: overall multiplier for the totals in EXP_RIDERSHIP
# - line_scales: per-line multipliers (e.g., CNE -> Lakeshore boost)
# - spread: random noise as fraction of per-interval mean
# - shape: "gaussian" or "beta"
# - peak_time_min/peak_width_min: Gaussian timing
# - beta_a/beta_b: Beta curve shape
# NOTE: These are intentionally different across 10 instances (Mon–Sun + Christmas + CNE + Valentines)
PROFILES = {
    "Monday":     {"seed": 101, "global_scale": 0.95, "spread": 0.10, "shape": "gaussian", "peak_time_min": 70, "peak_width_min": 26, "line_scales": {}},
    "Tuesday":    {"seed": 102, "global_scale": 1.05, "spread": 0.10, "shape": "gaussian", "peak_time_min": 75, "peak_width_min": 22, "line_scales": {}},
    "Wednesday":  {"seed": 103, "global_scale": 1.15, "spread": 0.10, "shape": "gaussian", "peak_time_min": 75, "peak_width_min": 22, "line_scales": {}},
    "Thursday":   {"seed": 104, "global_scale": 1.06, "spread": 0.10, "shape": "gaussian", "peak_time_min": 80, "peak_width_min": 24, "line_scales": {}},
    "Friday":     {"seed": 105, "global_scale": 0.88, "spread": 0.11, "shape": "gaussian", "peak_time_min": 60, "peak_width_min": 28, "line_scales": {}},
    "Saturday":   {"seed": 106, "global_scale": 0.55, "spread": 0.12, "shape": "gaussian", "peak_time_min": 90, "peak_width_min": 36, "line_scales": {}},
    "Sunday":     {"seed": 107, "global_scale": 0.50, "spread": 0.12, "shape": "gaussian", "peak_time_min": 85, "peak_width_min": 36, "line_scales": {}},
    "Christmas":  {"seed": 201, "global_scale": 1.2, "spread": 0.08, "shape": "gaussian", "peak_time_min": 60, "peak_width_min": 50, "line_scales": {}},
    "CNE":        {"seed": 202, "global_scale": 1.30, "spread": 0.2, "shape": "gaussian", "peak_time_min": 95, "peak_width_min": 22,
                   "line_scales": {'Lakeshore West': 1.60, 'Lakeshore East': 1.40}},
    "Valentines": {"seed": 203, "global_scale": 1.2, "spread": 0.2, "shape": "gaussian", "peak_time_min": 10, "peak_width_min": 26,
                   "line_scales": {'Lakeshore West': 1.03, 'Lakeshore East': 1.03}},
}

# ---------------------------
# HELPERS (simple procedural functions)
# ---------------------------

def time_labels(interval_size):
    labels = []
    for t in range(0, 120, interval_size):
        hour = 4 + (t // 60)
        minute = t % 60
        labels.append(f"{hour}:{minute:02d}")
    return labels

def gaussian_curve(num_points, peak_idx, width_pts):
    x = np.arange(num_points)
    curve = np.exp(-0.5 * ((x - peak_idx) / max(width_pts, 1e-6))**2)
    s = curve.sum()
    if s == 0:
        curve = np.ones(num_points)
        s = curve.sum()
    return curve / s

def beta_curve(num_points, a, b):
    x = np.linspace(0, 1, num_points)
    a = max(a, 0.1); b = max(b, 0.1)
    curve = (x**(a-1)) * ((1-x)**(b-1))
    curve[np.isnan(curve)] = 0
    s = curve.sum()
    if s == 0:
        curve = np.ones(num_points)
        s = curve.sum()
    return curve / s

def build_curve(shape, interval_size, peak_time_min=75, peak_width_min=24, beta_a=3.0, beta_b=3.0):
    num_points = 120 // interval_size
    if shape == "gaussian":
        peak_idx = int(round(peak_time_min / interval_size))
        width_pts = max(1, int(round(peak_width_min / interval_size)))
        return gaussian_curve(num_points, peak_idx, width_pts)
    elif shape == "beta":
        return beta_curve(num_points, beta_a, beta_b)
    else:
        raise ValueError("shape must be 'gaussian' or 'beta'")

# ---------------------------
# CORE GENERATION (simplified like the user's original)
# ---------------------------

def generate_scenario(seed=None, interval_size=INTERVAL_SIZE, global_scale=1.0, spread=0.10,
                      shape="gaussian", peak_time_min=75, peak_width_min=24, beta_a=3.0, beta_b=3.0,
                      line_scales=None):
    if seed is None:
        seed = time_ns()
    if not isinstance(seed, int):
        seed = sum(ord(c) for c in str(seed))
    np.random.seed(seed % (2**32 - 1))

    if line_scales is None:
        line_scales = {}

    labels = time_labels(interval_size)
    curve = build_curve(shape, interval_size, peak_time_min, peak_width_min, beta_a, beta_b)

    scenario = {}
    for line in EXP_RIDERSHIP:
        scenario[line] = {}

    for line, base_total in EXP_RIDERSHIP.items():
        lscale = line_scales.get(line, 1.0)
        target_total = base_total * global_scale * lscale

        mean_per_bucket = target_total * curve
        noise_std = np.maximum(1e-9, spread * mean_per_bucket)
        noisy = np.maximum(0.0, np.random.normal(loc=mean_per_bucket, scale=noise_std))

        s = noisy.sum()
        if s > 0:
            adjusted = noisy * (target_total / s)
        else:
            adjusted = np.ones_like(noisy) * (target_total / len(noisy))

        ints = np.floor(adjusted).astype(int)
        remainder = int(round(target_total - ints.sum()))
        if remainder > 0:
            # add to buckets with largest fractional parts
            remainders = (adjusted - ints).argsort()[::-1][:remainder]
            ints[remainders] += 1
        elif remainder < 0:
            # subtract from buckets with largest overshoot
            remainders = (ints - adjusted).argsort()[::-1][:(-remainder)]
            ints[remainders] -= 1

        for i, tlabel in enumerate(labels):
            scenario[line][tlabel] = int(max(0, ints[i]))

    return scenario

def get_instances(names=None):
    # Returns dict[name] -> scenario
    if names is None:
        names = list(PROFILES.keys())
    out = {}
    for name in names:
        p = PROFILES[name]
        out[name] = generate_scenario(
            seed=p["seed"],
            interval_size=INTERVAL_SIZE,
            global_scale=p["global_scale"],
            spread=p["spread"],
            shape=p["shape"],
            peak_time_min=p.get("peak_time_min", 75),
            peak_width_min=p.get("peak_width_min", 24),
            beta_a=p.get("beta_a", 3.0),
            beta_b=p.get("beta_b", 3.0),
            line_scales=p.get("line_scales", {})
        )
    return out

def save_jsons(instances, out_dir):
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)
    paths = []
    for name, scen in instances.items():
        path = out_dir / f"scenario_{name.lower().replace(' ', '_')}.json"
        with open(path, "w", encoding="utf-8") as f:
            json.dump(scen, f, indent=2)
        paths.append(str(path))
    return paths

In [4]:
instances=get_instances()


In [ ]:
scenarios = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday','Christmas','CNE','Valentines']
results = []
for day in scenarios:
    print(f"\n--------- {day} -----------")
    data = instances[day]

    line_map = {
        'Lakeshore West': 1,
        'Lakeshore East': 2,
        'Kitchener': 3,
        'Milton': 4,
        'Barrie': 5,
        'Stouffville': 6,
        'Richmond Hill': 7
    }

    time_map = {time: i + 1 for i, time in enumerate(data['Lakeshore West'].keys())}

    D = {}
    for line_name, time_dict in data.items():
        l = line_map[line_name]
        for time, d in time_dict.items():
            t = time_map[time]
            D[(l, t)] = d
            

    m = gp.Model()
    m.Params.outputFlag = 1
    m.Params.TimeLimit = 100

    lines = range(1, 8)
    platforms = range(1, 15)
    times = range(1, 25)

    x = m.addVars(lines, platforms, times, vtype=gp.GRB.INTEGER, lb=0, name="x")
    p = m.addVars(lines, platforms, times, vtype=gp.GRB.INTEGER, lb=0, name="p")
    y = m.addVars(lines, platforms, times, vtype=gp.GRB.BINARY, name="y")
    u = m.addVars(lines, times, vtype=gp.GRB.INTEGER, lb=0, name="u")

    m.setObjective(
        gp.quicksum(u[l, t] for l in lines for t in times),
        gp.GRB.MINIMIZE
    )
    m.addConstrs((u[l, 1] >= D.get((l, 1), 0) - gp.quicksum(p[l, tr, 1] for tr in platforms)
                  for l in lines), name="Starting Demand")

    m.addConstrs((u[l, t] >= D.get((l, t), 0) - gp.quicksum(p[l, tr, t] for tr in platforms) + u[l, t-1]
                  for l in lines 
                  for t in range(2, 25)), name="Demand Satisfaction")

    m.addConstrs((p[l, tr, t] <= 162 * x[l, tr, t] 
                  for l in lines 
                  for tr in platforms 
                  for t in times), name="Passenger Capacity")

    m.addConstrs((gp.quicksum(y[l, tr, t] for l in lines for tr in platforms) <= 14 
                  for t in times), name="Platform Usage")

    m.addConstrs((gp.quicksum(y[l, tr, t] for l in lines) <= 1
                  for tr in platforms
                  for t in times), name="Platform difference")

    m.addConstrs((x[l, tr, t] <= 12 * y[l, tr, t] 
                  for l in lines 
                  for tr in platforms 
                  for t in times), name="Platform Coach Link")

    m.addConstr( gp.quicksum(y[l, tr, t] for l in lines for tr in platforms for t in times) <= 80, name="Train Formation")

    m.addConstr(gp.quicksum(x[l, tr, t] for l in lines for tr in platforms for t in times) <= 756, name="Total Coach")


    m.optimize()
    
    coaches = sum(x[l, tr, t].x for l in lines for tr in platforms for t in times)


    print("Model Runtime:",m.runtime)
    print("Model Obj:",m.ObjVal)
    print("Total Coaches Used:", coaches)
    
    results.append({
        "Scenario": day,
        "Runtime (s)": m.Runtime,
        "Objective Value": m.ObjVal,
        "Total Coaches Used": coaches
    })
    
results_df = pd.DataFrame(results)

print(results_df)


--------- Monday -----------
Set parameter Username
Set parameter LicenseID to value 2734443
Academic license - for non-commercial use only - expires 2026-11-07
Set parameter OutputFlag to value 1
Set parameter TimeLimit to value 100
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 24.04.3 LTS")

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  100

Optimize a model with 5234 rows, 7224 columns and 21497 nonzeros
Model fingerprint: 0x8c20e937
Variable types: 0 continuous, 7224 integer (2352 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
Found heuristic solution: objective 1050333.0000
Presolve removed 2698 rows and 2674 columns
Presolve time: 0.07s
Presolved: 2536 rows, 4550 columns, 13797 nonzeros
Var

In [104]:
#Old
m = gp.Model("MIP")
m.Params.outputFlag = 1

lines = range(1, 8)
platforms = range(1, 15)
times = range(1, 25)

x = m.addVars(lines, platforms, times,
              vtype=gp.GRB.INTEGER,
              lb=0,
              name="x")

y = m.addVars(platforms, times,
              vtype=gp.GRB.BINARY,
              name="y")

u = m.addVars(lines, times,
              vtype=gp.GRB.INTEGER,
              lb=0,
              name="u")

m.setObjective(
    gp.quicksum(u[l, t] for l in lines for t in times),
    gp.GRB.MINIMIZE
)


m.addConstrs((u[l, t] >= D.get((l, t), 0) - 162 * gp.quicksum(x[l, tr, t] for tr in platforms)for l in lines for t in times),name="Demand Satisfaction")

m.addConstrs((gp.quicksum(y[tr, t] for tr in platforms) <= 14 for t in times), name="Platform Usage")
m.addConstrs((x[l, tr, t] <= 12 * y[tr, t] for l in lines for tr in platforms for t in times), name="Platform Coach Link")
m.addConstr(gp.quicksum(y[tr, t] for tr in platforms for t in times) <= 80, name="Train Formation Limit")

m.optimize()

print("Model var count:", m.NumVars)
 

Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700 CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2545 rows, 2856 columns and 7896 nonzeros
Model fingerprint: 0xcd705d5f
Variable types: 0 continuous, 2856 integer (336 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+00, 3e+03]
Found heuristic solution: objective 102444.00000
Presolve removed 2545 rows and 2856 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.01 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 0 102444 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e

In [105]:
m.display()

Minimize
u[1,1] + u[1,2] + u[1,3] + u[1,4] + u[1,5] + u[1,6] + u[1,7] + u[1,8] + u[1,9]
+ u[1,10] + u[1,11] + u[1,12] + u[1,13] + u[1,14] + u[1,15] + u[1,16] + u[1,17]
+ u[1,18] + u[1,19] + u[1,20] + u[1,21] + u[1,22] + u[1,23] + u[1,24] + u[2,1] + u[2,2]
+ u[2,3] + u[2,4] + u[2,5] + u[2,6] + u[2,7] + u[2,8] + u[2,9] + u[2,10] + u[2,11]
+ u[2,12] + u[2,13] + u[2,14] + u[2,15] + u[2,16] + u[2,17] + u[2,18] + u[2,19]
+ u[2,20] + u[2,21] + u[2,22] + u[2,23] + u[2,24] + u[3,1] + u[3,2] + u[3,3] + u[3,4]
+ u[3,5] + u[3,6] + u[3,7] + u[3,8] + u[3,9] + u[3,10] + u[3,11] + u[3,12] + u[3,13]
+ u[3,14] + u[3,15] + u[3,16] + u[3,17] + u[3,18] + u[3,19] + u[3,20] + u[3,21]
+ u[3,22] + u[3,23] + u[3,24] + u[4,1] + u[4,2] + u[4,3] + u[4,4] + u[4,5] + u[4,6]
+ u[4,7] + u[4,8] + u[4,9] + u[4,10] + u[4,11] + u[4,12] + u[4,13] + u[4,14] + u[4,15]
+ u[4,16] + u[4,17] + u[4,18] + u[4,19] + u[4,20] + u[4,21] + u[4,22] + u[4,23]
+ u[4,24] + u[5,1] + u[5,2] + u[5,3] + u[5,4] + u[5,5] + u[5,6] + u[5,7] + u[5,

C:\Users\JacksonWang\AppData\Local\Temp\ipykernel_28912\137993151.py:1: DeprecationWarning: Model.display() is deprecated
  m.display()


x[3,4,9] + 162.0 x[3,5,9] + 162.0 x[3,6,9] + 162.0 x[3,7,9] + 162.0 x[3,8,9] + 162.0
x[3,9,9] + 162.0 x[3,10,9] + 162.0 x[3,11,9] + 162.0 x[3,12,9] + 162.0 x[3,13,9] + 162.0
 x[3,14,9] + u[3,9] >= 501
Demand Satisfaction[3,10]: 162.0 x[3,1,10] + 162.0 x[3,2,10] + 162.0 x[3,3,10] + 162.0
x[3,4,10] + 162.0 x[3,5,10] + 162.0 x[3,6,10] + 162.0 x[3,7,10] + 162.0 x[3,8,10] +
162.0 x[3,9,10] + 162.0 x[3,10,10] + 162.0 x[3,11,10] + 162.0 x[3,12,10] + 162.0
 x[3,13,10] + 162.0 x[3,14,10] + u[3,10] >= 815
Demand Satisfaction[3,11]: 162.0 x[3,1,11] + 162.0 x[3,2,11] + 162.0 x[3,3,11] + 162.0
x[3,4,11] + 162.0 x[3,5,11] + 162.0 x[3,6,11] + 162.0 x[3,7,11] + 162.0 x[3,8,11] +
162.0 x[3,9,11] + 162.0 x[3,10,11] + 162.0 x[3,11,11] + 162.0 x[3,12,11] + 162.0
 x[3,13,11] + 162.0 x[3,14,11] + u[3,11] >= 730
Demand Satisfaction[3,12]: 162.0 x[3,1,12] + 162.0 x[3,2,12] + 162.0 x[3,3,12] + 162.0
x[3,4,12] + 162.0 x[3,5,12] + 162.0 x[3,6,12] + 162.0 x[3,7,12] + 162.0 x[3,8,12] +
162.0 x[3,9,12] + 162.0 x[3,

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
# for tr in platforms:
#     for t in times:
#         print(f"y[{tr},{t}] = {y[tr, t].X}")

for l in lines:
    for tr in platforms:
        for t in times:
            print(f"y[{l},{tr},{t}] = {y[l, tr, t].X}")

y[1,1,1] = -0.0
y[1,1,2] = -0.0
y[1,1,3] = -0.0
y[1,1,4] = -0.0
y[1,1,5] = -0.0
y[1,1,6] = -0.0
y[1,1,7] = -0.0
y[1,1,8] = -0.0
y[1,1,9] = 1.0
y[1,1,10] = 1.0
y[1,1,11] = -0.0
y[1,1,12] = -0.0
y[1,1,13] = -0.0
y[1,1,14] = 0.0
y[1,1,15] = -0.0
y[1,1,16] = -0.0
y[1,1,17] = -0.0
y[1,1,18] = -0.0
y[1,1,19] = -0.0
y[1,1,20] = -0.0
y[1,1,21] = -0.0
y[1,1,22] = -0.0
y[1,1,23] = -0.0
y[1,1,24] = 0.0
y[1,2,1] = -0.0
y[1,2,2] = -0.0
y[1,2,3] = -0.0
y[1,2,4] = -0.0
y[1,2,5] = -0.0
y[1,2,6] = -0.0
y[1,2,7] = 1.0
y[1,2,8] = -0.0
y[1,2,9] = -0.0
y[1,2,10] = -0.0
y[1,2,11] = -0.0
y[1,2,12] = -0.0
y[1,2,13] = -0.0
y[1,2,14] = -0.0
y[1,2,15] = -0.0
y[1,2,16] = -0.0
y[1,2,17] = -0.0
y[1,2,18] = -0.0
y[1,2,19] = -0.0
y[1,2,20] = -0.0
y[1,2,21] = -0.0
y[1,2,22] = -0.0
y[1,2,23] = 0.0
y[1,2,24] = -0.0
y[1,3,1] = 1.0
y[1,3,2] = 1.0
y[1,3,3] = -0.0
y[1,3,4] = -0.0
y[1,3,5] = -0.0
y[1,3,6] = -0.0
y[1,3,7] = -0.0
y[1,3,8] = 1.0
y[1,3,9] = -0.0
y[1,3,10] = -0.0
y[1,3,11] = -0.0
y[1,3,12] = -0.0
y[1,3,13] = -0.0

In [107]:
for l in lines:
    for t in times:
        print(f"u[{l},{t}] = {u[l, t].X}")

u[1,1] = 0.0
u[1,2] = 0.0
u[1,3] = 0.0
u[1,4] = 0.0
u[1,5] = 0.0
u[1,6] = 0.0
u[1,7] = 0.0
u[1,8] = 0.0
u[1,9] = 0.0
u[1,10] = 0.0
u[1,11] = 0.0
u[1,12] = 0.0
u[1,13] = -0.0
u[1,14] = -0.0
u[1,15] = -0.0
u[1,16] = -0.0
u[1,17] = -0.0
u[1,18] = -0.0
u[1,19] = -0.0
u[1,20] = 0.0
u[1,21] = 0.0
u[1,22] = 0.0
u[1,23] = 0.0
u[1,24] = 0.0
u[2,1] = 0.0
u[2,2] = 0.0
u[2,3] = 0.0
u[2,4] = 0.0
u[2,5] = 0.0
u[2,6] = 0.0
u[2,7] = 0.0
u[2,8] = 0.0
u[2,9] = 0.0
u[2,10] = 0.0
u[2,11] = 0.0
u[2,12] = 0.0
u[2,13] = -0.0
u[2,14] = -0.0
u[2,15] = -0.0
u[2,16] = -0.0
u[2,17] = -0.0
u[2,18] = -0.0
u[2,19] = -0.0
u[2,20] = 0.0
u[2,21] = 0.0
u[2,22] = 0.0
u[2,23] = 0.0
u[2,24] = 0.0
u[3,1] = 0.0
u[3,2] = 0.0
u[3,3] = 0.0
u[3,4] = 0.0
u[3,5] = 0.0
u[3,6] = 0.0
u[3,7] = 0.0
u[3,8] = 0.0
u[3,9] = 0.0
u[3,10] = 0.0
u[3,11] = 0.0
u[3,12] = 0.0
u[3,13] = -0.0
u[3,14] = -0.0
u[3,15] = -0.0
u[3,16] = -0.0
u[3,17] = -0.0
u[3,18] = -0.0
u[3,19] = -0.0
u[3,20] = 0.0
u[3,21] = 0.0
u[3,22] = 0.0
u[3,23] = 0.0
u[3,24] = 0.

In [108]:
for l in lines:
    for tr in platforms:
        for t in times:
            if x[l, tr, t].X > 0:   
                print(f"x[{l},{tr},{t}] = {x[l, tr, t].X}")

x[1,1,19] = 12.0
x[1,2,17] = 12.0
x[1,3,1] = 12.0
x[1,3,8] = 12.0
x[1,3,14] = 12.0
x[1,4,9] = 12.0
x[1,4,13] = 12.0
x[1,4,14] = 12.0
x[1,4,15] = 12.0
x[1,4,16] = 12.0
x[1,5,4] = 12.0
x[1,5,13] = 12.0
x[1,5,16] = 12.0
x[1,6,18] = 12.0
x[1,7,18] = 12.0
x[1,7,23] = 12.0
x[1,8,3] = 12.0
x[1,8,5] = 12.0
x[1,8,10] = 12.0
x[1,8,17] = 12.0
x[1,9,6] = 12.0
x[1,9,12] = 12.0
x[1,9,21] = 12.0
x[1,10,2] = 12.0
x[1,11,7] = 12.0
x[1,11,11] = 12.0
x[1,11,24] = 12.0
x[1,13,15] = 12.0
x[1,13,19] = 12.0
x[1,13,20] = 12.0
x[1,13,22] = 12.0
x[1,14,12] = 12.0
x[2,1,19] = 12.0
x[2,2,17] = 12.0
x[2,3,1] = 12.0
x[2,3,8] = 12.0
x[2,3,14] = 12.0
x[2,4,9] = 12.0
x[2,4,13] = 12.0
x[2,4,14] = 12.0
x[2,4,15] = 12.0
x[2,4,16] = 12.0
x[2,5,4] = 12.0
x[2,5,13] = 12.0
x[2,5,16] = 12.0
x[2,6,18] = 12.0
x[2,7,18] = 12.0
x[2,7,23] = 12.0
x[2,8,3] = 12.0
x[2,8,5] = 12.0
x[2,8,10] = 12.0
x[2,8,17] = 12.0
x[2,9,6] = 12.0
x[2,9,12] = 12.0
x[2,9,21] = 12.0
x[2,10,2] = 12.0
x[2,11,7] = 12.0
x[2,11,11] = 12.0
x[2,11,24] = 12.0
x[

In [8]:
instances

{'Monday': {'Lakeshore West': {'4:00': 64,
   '4:05': 92,
   '4:10': 155,
   '4:15': 236,
   '4:20': 365,
   '4:25': 485,
   '4:30': 644,
   '4:35': 1008,
   '4:40': 984,
   '4:45': 1649,
   '4:50': 1936,
   '4:55': 1990,
   '5:00': 2381,
   '5:05': 2293,
   '5:10': 2295,
   '5:15': 2719,
   '5:20': 2382,
   '5:25': 2533,
   '5:30': 2317,
   '5:35': 1641,
   '5:40': 1270,
   '5:45': 1111,
   '5:50': 584,
   '5:55': 443},
  'Lakeshore East': {'4:00': 39,
   '4:05': 70,
   '4:10': 113,
   '4:15': 179,
   '4:20': 289,
   '4:25': 394,
   '4:30': 584,
   '4:35': 766,
   '4:40': 914,
   '4:45': 1108,
   '4:50': 1299,
   '4:55': 1730,
   '5:00': 1802,
   '5:05': 2304,
   '5:10': 2022,
   '5:15': 2322,
   '5:20': 1642,
   '5:25': 1682,
   '5:30': 1271,
   '5:35': 1198,
   '5:40': 1060,
   '5:45': 730,
   '5:50': 547,
   '5:55': 416},
  'Kitchener': {'4:00': 27,
   '4:05': 36,
   '4:10': 59,
   '4:15': 97,
   '4:20': 156,
   '4:25': 213,
   '4:30': 333,
   '4:35': 469,
   '4:40': 501,
   '4:45'

In [17]:
cumul_demand = []

for scenario, lines in instances.items():
    total_demand = 0
    for times in lines.values():
        for acc_demand in times.values():
            total_demand += acc_demand
    cumul_demand.append((scenario, total_demand))
        

In [18]:
cumul_demand

[('Monday', 102444),
 ('Tuesday', 113228),
 ('Wednesday', 124011),
 ('Thursday', 114305),
 ('Friday', 94895),
 ('Saturday', 59310),
 ('Sunday', 53917),
 ('Christmas', 129403),
 ('CNE', 179513),
 ('Valentines', 131526)]

In [19]:
results_df

,Scenario,Runtime (s),Objective Value,Total Coaches Used
0,Monday,6.351891,34135.0,670.000000
1,Tuesday,6.202902,29500.0,737.000000
2,Wednesday,100.053221,35932.0,756.000000
3,Thursday,10.176706,35544.0,713.000000
4,Friday,8.379468,35924.0,685.000001
5,Saturday,8.426135,17537.0,413.000000
6,Sunday,8.721174,16929.0,476.000000
7,Christmas,100.041086,65466.0,756.000000
8,CNE,100.077712,156052.0,756.000000
9,Valentines,100.039873,121148.0,756.000000


In [15]:
for lines in instances.values():
    for times in lines.values():
        for time in times.values():
            print(time)

64
92
155
236
365
485
644
1008
984
1649
1936
1990
2381
2293
2295
2719
2382
2533
2317
1641
1270
1111
584
443
39
70
113
179
289
394
584
766
914
1108
1299
1730
1802
2304
2022
2322
1642
1682
1271
1198
1060
730
547
416
27
36
59
97
156
213
333
469
501
815
730
1004
1086
1342
1091
1136
1117
1045
820
821
496
394
321
208
17
40
62
71
151
224
252
430
421
584
724
888
972
1015
927
1146
998
801
694
701
525
346
271
221
11
21
41
53
106
119
162
221
331
394
454
572
575
681
721
688
558
617
500
445
362
244
203
125
12
16
34
50
64
116
149
257
281
422
441
432
538
568
615
468
509
445
421
350
284
227
154
121
7
12
22
34
46
90
94
119
203
239
282
292
297
358
372
367
348
351
242
183
175
117
75
85
4
9
20
37
94
165
274
447
776
916
1825
2198
2610
2851
3325
4012
2918
3250
2615
2215
1904
1249
726
461
2
6
10
31
66
119
208
377
642
1176
1164
1603
1656
2725
2720
2512
2922
2007
2200
1543
1427
937
643
361
1
4
10
17
34
79
134
235
331
610
679
1180
1250
1486
1614
1327
1382
1303
1210
1027
739
531
389
253
1
3
7
17
26
53
112
198
29